In [1]:
import requests
from bs4 import BeautifulSoup

url = "https://www.mayoclinic.org/diseases-conditions/index?letter=A"
headers = {"User-Agent": "Mozilla/5.0"}

response = requests.get(url, headers=headers)
if response.status_code == 200:
    soup = BeautifulSoup(response.text, "html.parser")
    diseases = soup.select(".index-content a")
    
    disease_list = [disease.get_text(strip=True) for disease in diseases]
    
    print("Diseases starting with 'A':")
    for disease in disease_list:
        print(disease)
else:
    print("Failed to retrieve data")


Diseases starting with 'A':


In [1]:
import requests
from bs4 import BeautifulSoup

In [ ]:
import requests
from bs4 import BeautifulSoup
import csv
from tqdm import tqdm

# Base URL for diseases
base_url = "https://www.mayoclinic.org/diseases-conditions/index?letter=A"

# Send HTTP request
response = requests.get(base_url)
if response.status_code != 200:
    print("Failed to retrieve webpage")
    exit()

soup = BeautifulSoup(response.text, "html.parser")

items = soup.select(".cmp-results-with-primary-name__see-link, .cmp-results-with-primary-name a")

# Extract diseases and links
disease_data = []
for item in tqdm(items, desc = "Scraping diseases....."):
    disease_name = item.text.strip()
    disease_link = f"https://www.mayoclinic.org{item['href']}" if item['href'].startswith("/") else item['href']
    disease_data.append([disease_name, disease_link])

# Save to CSV
with open("mayo_diseases.csv", "w", newline="", encoding="utf-8") as file:
    writer = csv.writer(file)
    writer.writerow(["Disease", "Link"])
    writer.writerows(disease_data)

print("Scraping complete! Data saved in mayo_diseases.csv")


Scraping diseases.....: 100%|██████████| 133/133 [00:00<?, ?it/s]

Scraping complete! Data saved in mayo_diseases.csv


In [ ]:
def web_Scrapping(base_url):
    response = requests.get(base_url)
    if response.status_code != 200:
        print("Failed to retrieve page")
        exit()
        
    soup = BeautifulSoup(response.text, "html.parser")
    
    items = soup.select(".cmp-results-with-primary-name__see-link, .cmp-results-with-primary-name a")
    
    
    disease_data = []
    for item in tqdm(items, desc="Scrapping Diseases"):
        disease_name = item.text.strip()
        disease_link = f"https://www.mayoclinic.org{item['href']}" if item["href"].startswith("/") else item['href']
        disease_data.append([disease_name, disease_link])
        
    file_exists = os.path.isfile("mayo_diseases.csv")
        
    with open("mayo_diseases.csv", "a", newline="", encoding="utf-8") as file:
        writer = csv.writer(file)
        if not file_exists:
            writer.writerow(["Disease", "Link"])
        writer.writerows(disease_data)
        
    print("Scraping Completed! Data Saved")
    
    
    
    

In [21]:
import string
import os

In [26]:
for letter in string.ascii_uppercase:
    base_url = f"https://www.mayoclinic.org/diseases-conditions/index?letter={letter}"
    web_Scrapping(base_url)
print("done")

Scrapping Diseases: 100%|██████████| 133/133 [00:00<?, ?it/s]


Scraping Completed! Data Saved


Scrapping Diseases: 100%|██████████| 38/38 [00:00<?, ?it/s]


Scraping Completed! Data Saved


Scrapping Diseases: 100%|██████████| 163/163 [00:00<?, ?it/s]


...

Scraping Completed! Data Saved


Scrapping Diseases: 100%|██████████| 1/1 [00:00<?, ?it/s]


Scraping Completed! Data Saved


Scrapping Diseases: 100%|██████████| 1/1 [00:00<?, ?it/s]

Scraping Completed! Data Saved
done


In [2]:
import pandas as pd

In [4]:
df = pd.read_csv("mayo_diseases.csv")

'https://www.mayoclinic.org/diseases-conditions/atrial-fibrillation/symptoms-causes/syc-20350624'

In [7]:
import requests
from bs4 import BeautifulSoup
from typing import Dict
import logging

def scrape_medical_info(url: str) -> Dict[str, str]:
    """
    Scrapes medical information from Mayo Clinic website.
    
    Args:
        url (str): Mayo Clinic URL to scrape
        
    Returns:
        Dict[str, str]: Dictionary containing medical information by section
    """
    # Setup logging
    logging.basicConfig(level=logging.INFO)
    logger = logging.getLogger(__name__)

    # Headers to mimic browser request
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36",
        "Accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/webp,*/*;q=0.8"
    }

    sections = [
        "Overview", "Symptoms", "When to see a doctor", 
        "Causes", "Risk factors", "Complications", "Prevention"
    ]
    
    content = {}

    try:
        # Fetch webpage
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
        
        # Parse HTML
        soup = BeautifulSoup(response.text, 'html.parser')
        
        # Extract content for each section
        for section in sections:
            section_content = []
            
            # Find section header
            header = soup.find(['h2', 'h3'], string=section)
            if header:
                # Get all text elements after header until next section
                for elem in header.find_next_siblings(['p', 'ul', 'li']):
                    if elem.name in ['h2', 'h3']:
                        break
                    section_content.append(elem.get_text(strip=True))
                    
            content[section] = " ".join(section_content) if section_content else "No data found"
            
        return content

    except requests.exceptions.RequestException as e:
        logger.error(f"Failed to fetch URL: {e}")
        return {section: "Error fetching data" for section in sections}
    
    except Exception as e:
        logger.error(f"Unexpected error: {e}")
        return {section: "Error processing data" for section in sections}

# Example usage
if __name__ == "__main__":
    url = df["Link"][1]
    data = scrape_medical_info(url)
    
    # Print results
    for section, text in data.items():
        print(f"\n{section}:")
        print(f"{text}\n")
        print("-" * 80)


Overview:
Hyperhidrosis (hi-pur-hi-DROE-sis) is excessive sweating that's not always related to heat or exercise. You may sweat so much that it soaks through your clothes or drips off your hands. Heavy sweating can disrupt your day and cause social anxiety and embarrassment. Hyperhidrosis treatment usually helps. It often begins with antiperspirants. If these don't help, you may need to try different medications and therapies. In severe cases, your health care provider may suggest surgery to remove the sweat glands or to disconnect the nerves related to producing too much sweat. Sometimes an underlying condition may be found and treated. The main symptom of hyperhidrosis is heavy sweating. This goes beyond the sweating from being in a hot environment, exercising, or feeling anxious or stressed. The type of hyperhidrosis that usually affects the hands, feet, underarms or face causes at least one episode a week when you're awake. And the sweating usually happens on both sides of the bod

In [9]:
df["Link"][2]

'https://www.mayoclinic.org/diseases-conditions/bartholin-cyst/symptoms-causes/syc-20369976'